In [2]:
from classes import *
from collections import defaultdict



In [ ]:
def reweave(mergers: list[Alignment], graph: Pangraph) -> tuple[list[MergePromise], Pangraph]:

    # assign id to alignments
    for i, a in enumerate(mergers):
        a.id = i

    # extract targeted blocks and corresponding mergers
    TB = target_blocks(mergers)

    # for every block, split it according to the mergers
    U, H = [], []
    for bid, M in TB.items(): # can be done in parallel: the graph and block are not modified at this stage
        graph_update, half_promises = split_block(bid, M, graph)
        U.append(graph_update)
        H += half_promises

    # group and create merge promises
    merge_promises = format_promises(H)

    for u in U:
        #  inplace graph update
        graph.update(u)


    return graph, merge_promises